<a href="https://colab.research.google.com/github/senthilkumar-dimitra/NDVI-time-series-analysis/blob/expts/sugarcane_farm_veg_indices_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap -q
!pip install pykalman -q
!pip install PyWavelets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.0 MB/s eta 0:00:00


In [2]:
import ee
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from scipy.signal import savgol_filter
import statsmodels.api as sm
from scipy.ndimage import gaussian_filter1d
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline, BSpline, splrep, splev
from scipy.fft import fft, ifft
import pywt

pd.set_option('display.max_rows', None)

# Authenticate and initialize the library
ee.Authenticate()
ee.Initialize(project='senthilkumar-dimitra')

In [ ]:
# Define the coordinates for the AOI (single polygon)
coordinates = [(11.9339692, 9.6092636), (11.9378834, 9.607271), (11.937092, 9.590627), (11.9297, 9.591299)]  # Dangote sugarcane field co-ordinates in Nigeria

# Define the time range
start_date = '2018-12-31'
end_date = '2024-01-01'

# Function to mask clouds based on the QA60 band of Sentinel-2
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(cloud_mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Function to calculate RECI
def calculate_reci(image):
    reci = image.expression(
        'B8 / B5 - 1', {
            'B8': image.select('B8'),  # NIR
            'B5': image.select('B5')   # Red-Edge
        }).rename('RECI')
    return image.addBands(reci)

# Function to calculate LSWI
def calculate_lswi(image):
    lswi = image.normalizedDifference(['B8', 'B11']).rename('LSWI')  # NIR and SWIR
    return image.addBands(lswi)

# Load Sentinel-2 data, filter by date and bounds, apply cloud mask, and calculate indices
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .map(mask_clouds) \
    .map(calculate_ndvi) \
    .map(calculate_reci) \
    .map(calculate_lswi)

# Define the polygon as an ee.Geometry
aoi = ee.Geometry.Polygon([coordinates])

# Calculate mean RECI, LSWI, and NDVI for the polygon over time
def get_index_timeseries(aoi, index_name):
    def compute_index(image):
        mean_index = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=30
        ).get(index_name)
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'date': date, index_name: mean_index})

    index_collection = collection.filterBounds(aoi).select([index_name]).map(compute_index)
    return index_collection.getInfo()

# Get the NDVI, RECI, and LSWI time series
ndvi_timeseries = get_index_timeseries(aoi, 'NDVI')
reci_timeseries = get_index_timeseries(aoi, 'RECI')
lswi_timeseries = get_index_timeseries(aoi, 'LSWI')

# Convert the time series to DataFrames
def extract_index_data(index_timeseries, index_name):
    dates = []
    index_values = []

    for feature in index_timeseries['features']:
        properties = feature['properties']
        date = properties.get('date')
        index_value = properties.get(index_name)
        if date and index_value is not None:
            dates.append(date)
            index_values.append(index_value)

    return pd.DataFrame({'Date': pd.to_datetime(dates), index_name: index_values})

# Create DataFrames for NDVI, RECI, and LSWI
ndvi_df = extract_index_data(ndvi_timeseries, 'NDVI')
reci_df = extract_index_data(reci_timeseries, 'RECI')
lswi_df = extract_index_data(lswi_timeseries, 'LSWI')

# Merge the DataFrames on the Date
merged_df = pd.merge(pd.merge(ndvi_df, reci_df, on='Date'), lswi_df, on='Date')

# Apply Savitzky-Golay filter to smooth the indices
merged_df['NDVI_Smoothed'] = savgol_filter(merged_df['NDVI'], window_length=11, polyorder=2)
merged_df['RECI_Smoothed'] = savgol_filter(merged_df['RECI'], window_length=11, polyorder=2)
merged_df['LSWI_Smoothed'] = savgol_filter(merged_df['LSWI'], window_length=11, polyorder=2)

In [ ]:
# Fit linear regression models to detect trends for NDVI, RECI, and LSWI
merged_df['Date_ordinal'] = merged_df['Date'].map(pd.Timestamp.toordinal)

# NDVI trend
X_ndvi = sm.add_constant(merged_df['Date_ordinal'])
model_ndvi = sm.OLS(merged_df['NDVI_Smoothed'], X_ndvi).fit()
merged_df['NDVI_Trend'] = model_ndvi.predict(X_ndvi)

# RECI trend
X_reci = sm.add_constant(merged_df['Date_ordinal'])
model_reci = sm.OLS(merged_df['RECI_Smoothed'], X_reci).fit()
merged_df['RECI_Trend'] = model_reci.predict(X_reci)

# LSWI trend
X_lswi = sm.add_constant(merged_df['Date_ordinal'])
model_lswi = sm.OLS(merged_df['LSWI_Smoothed'], X_lswi).fit()
merged_df['LSWI_Trend'] = model_lswi.predict(X_lswi)

In [ ]:
merged_df

,Date,NDVI,RECI,LSWI,NDVI_Smoothed,RECI_Smoothed,LSWI_Smoothed,Date_ordinal,NDVI_Trend,RECI_Trend,LSWI_Trend
0,2019-01-07,0.314894,0.513942,0.129045,0.416816,0.717166,0.101015,737066,0.415369,0.863115,0.059882
1,2019-01-12,0.449595,0.773928,0.081498,0.404893,0.690400,0.099316,737071,0.415420,0.863330,0.060353
2,2019-01-17,0.445153,0.795077,0.091194,0.388342,0.655531,0.092459,737076,0.415471,0.863545,0.060824
3,2019-01-22,0.418231,0.696943,0.053699,0.367163,0.612560,0.080444,737081,0.415522,0.863760,0.061295
4,2019-01-27,0.390681,0.657087,0.030061,0.341356,0.561485,0.063272,737086,0.415573,0.863976,0.061766
5,2019-02-01,0.373572,0.597443,0.024808,0.310921,0.502307,0.040942,737091,0.415624,0.864191,0.062237
6,2019-02-11,0.132715,0.174045,0.092960,0.246839,0.378821,0.014562,737101,0.415726,0.864621,0.063179
7,2019-02-16,0.188303,0.264031,0.050843,0.199492,0.285534,-0.024366,737106,0.415777,0.864836,0.063650
8,2019-03-08,0.142024,0.202231,-0.107486,0.164381,0.221949,-0.068578,737126,0.415981,0.865697,0.065534
9,2019-03-13,0.154459,0.187321,-0.169760,0.129833,0.162891,-0.076665,737131,0.416032,0.865913,0.066004


In [ ]:
px.box(merged_df, y=['NDVI', 'RECI', 'LSWI'], title='Box Plot of NDVI, RECI, LSWI - Original from 2019-2023')

In [ ]:
import plotly.figure_factory as ff
ff.create_distplot([merged_df['NDVI'], merged_df['RECI'], merged_df['LSWI']], ['NDVI', 'RECI', 'LSWI'], bin_size=[.1, .25, .1]).update_layout(title='Distribution of NDVI, RECI, and LSWI')

## Raw vs Smoothed graphs comparison 2019 - 2023

In [ ]:
fig = go.Figure()

# Add original NDVI values and smoothed NDVI values
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['NDVI'], mode='markers+lines', name='NDVI (Original)'))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['NDVI_Smoothed'], mode='markers+lines', name='NDVI (Smoothed)'))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['NDVI_Trend'], mode='lines', name='NDVI Trend', line=dict(color='red')))

# Add RECI values and trends
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['RECI'], mode='markers+lines', name='RECI (Original)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['RECI_Smoothed'], mode='markers+lines', name='RECI (Smoothed)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['RECI_Trend'], mode='lines', name='RECI Trend', line=dict(color='darkgreen')))

# Add LSWI values and trends
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['LSWI'], mode='markers+lines', name='LSWI (Original)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['LSWI_Smoothed'], mode='markers+lines', name='LSWI (Smoothed)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['LSWI_Trend'], mode='lines', name='LSWI Trend', line=dict(color='darkblue')))

fig.update_layout(title='NDVI, RECI, and LSWI Trend Analysis (2019-2023)', xaxis_title='Date', yaxis_title='Index Value')
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## 2020 comparison of all 3 indices

In [ ]:
# Filter the DataFrame for a single year (e.g., 2020)
single_year_df = merged_df[(merged_df['Date'] >= '2020-01-01') & (merged_df['Date'] < '2021-01-01')]

fig = go.Figure()

# Add NDVI, RECI, and LSWI for the selected year
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI'))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['RECI'], mode='markers+lines', name='RECI', line=dict(color='green')))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['LSWI'], mode='markers+lines', name='LSWI', line=dict(color='blue')))

fig.update_layout(title='NDVI, RECI, and LSWI for 2020', xaxis_title='Date', yaxis_title='Index Value',legend_title_text = 'Index')
fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Scatter Plot
fig_scatter = px.scatter(
    single_year_df,
    x='Date',
    y=['NDVI', 'RECI', 'LSWI'],
    title='Scatter Plot of NDVI, RECI, and LSWI for 2020',
    labels={'value': 'Index Value', 'variable': 'Index'},
    color_discrete_map={'NDVI': 'green', 'RECI': 'orange', 'LSWI': 'blue'},
)

fig_scatter.update_traces(mode='markers+lines')
fig_scatter.show()

# Box Plot
fig_box = go.Figure()

# Add NDVI box plot
fig_box.add_trace(go.Box(
    y=merged_df['NDVI'],
    name='NDVI',
    marker_color='green'
))

# Add RECI box plot
fig_box.add_trace(go.Box(
    y=merged_df['RECI'],
    name='RECI',
    marker_color='orange'
))

# Add LSWI box plot
fig_box.add_trace(go.Box(
    y=merged_df['LSWI'],
    name='LSWI',
    marker_color='blue'
))

fig_box.update_layout(
    title='Box Plots of NDVI, RECI, and LSWI - 2019 to 2023',
    yaxis_title='Index Value',
    xaxis_title='Index',
    boxmode='group'  # Group boxes together
)

fig_box.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Dangote Sugarcane farm 1 vs farm 2 indices comparison

In [ ]:
# Define the coordinates for the AOI (single polygon)
coordinates = [(11.9339692, 9.6092636), (11.9378834, 9.607271), (11.937092, 9.590627), (11.9297, 9.591299)]  # Dangote sugarcane field co-ordinates in Nigeria
farm2_coordinates = [(11.876698, 9.611602), (11.885893, 9.611618), (11.885875, 9.600578), (11.876751, 9.600584)] # Dangote sugarcane field2

# Define the time range
start_date = '2019-01-01'
end_date = '2023-12-31'

# Function to mask clouds based on the QA60 band of Sentinel-2
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(cloud_mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Function to calculate RECI
def calculate_reci(image):
    reci = image.expression(
        'B8 / B5 - 1', {
            'B8': image.select('B8'),  # NIR
            'B5': image.select('B5')   # Red-Edge
        }).rename('RECI')
    return image.addBands(reci)

# Function to calculate LSWI
def calculate_lswi(image):
    lswi = image.normalizedDifference(['B8', 'B11']).rename('LSWI')  # NIR and SWIR
    return image.addBands(lswi)

# Load Sentinel-2 data, filter by date and bounds, apply cloud mask, and calculate indices
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .map(mask_clouds) \
    .map(calculate_ndvi) \
    .map(calculate_reci) \
    .map(calculate_lswi)

# Define the polygon as an ee.Geometry
aoi = ee.Geometry.Polygon([coordinates])

# Calculate mean RECI, LSWI, and NDVI for the polygon over time
def get_index_timeseries(aoi, index_name):
    def compute_index(image):
        mean_index = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=30
        ).get(index_name)
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'date': date, index_name: mean_index})

    index_collection = collection.filterBounds(aoi).select([index_name]).map(compute_index)
    return index_collection.getInfo()

# Get the NDVI, RECI, and LSWI time series
ndvi_timeseries = get_index_timeseries(aoi, 'NDVI')
reci_timeseries = get_index_timeseries(aoi, 'RECI')
lswi_timeseries = get_index_timeseries(aoi, 'LSWI')

# Convert the time series to DataFrames
def extract_index_data(index_timeseries, index_name):
    dates = []
    index_values = []

    for feature in index_timeseries['features']:
        properties = feature['properties']
        date = properties.get('date')
        index_value = properties.get(index_name)
        if date and index_value is not None:
            dates.append(date)
            index_values.append(index_value)

    return pd.DataFrame({'Date': pd.to_datetime(dates), index_name: index_values})

# Create DataFrames for NDVI, RECI, and LSWI
ndvi_df = extract_index_data(ndvi_timeseries, 'NDVI')
reci_df = extract_index_data(reci_timeseries, 'RECI')
lswi_df = extract_index_data(lswi_timeseries, 'LSWI')

# Merge the DataFrames on the Date
merged_df = pd.merge(pd.merge(ndvi_df, reci_df, on='Date'), lswi_df, on='Date')

#----------------------------------------
# Function to filter the DataFrame based on the 75th percentile within each slice
def filter_by_percentile_slicing(df, column, start_date, end_date, percentile=0.75):
    # Slice the DataFrame based on the date range
    df_slice = df[(df['Date'] >= start_date) & (df['Date'] < end_date)]

    # Calculate the 75th percentile for the specified column in this slice
    upper_bound = df_slice[column].quantile(percentile)

    # Filter the slice to keep only the values above or equal to the 75th percentile
    return df_slice[df_slice[column] >= upper_bound]

# Define the slicing periods (example: every 3 months)
slicing_periods = pd.date_range(start='2019-01-01', end='2023-12-31', freq='3M')

# Create an empty DataFrame to accumulate the filtered results
filtered_df_list = []

# Loop over the slicing periods and apply the filtering
for i in range(len(slicing_periods) - 1):
    start_date = slicing_periods[i]
    end_date = slicing_periods[i + 1]

    # Apply filtering for each index within the specified date range
    ndvi_filtered = filter_by_percentile_slicing(merged_df, 'NDVI', start_date, end_date, percentile=0.75)
    reci_filtered = filter_by_percentile_slicing(merged_df, 'RECI', start_date, end_date, percentile=0.75)
    lswi_filtered = filter_by_percentile_slicing(merged_df, 'LSWI', start_date, end_date, percentile=0.75)

    # Merge the filtered results by Date
    filtered_slice = pd.merge(pd.merge(ndvi_filtered[['Date', 'NDVI']],
                                       reci_filtered[['Date', 'RECI']], on='Date'),
                              lswi_filtered[['Date', 'LSWI']], on='Date')

    # Append the filtered slice to the list
    filtered_df_list.append(filtered_slice)

# Concatenate all the filtered slices into a final DataFrame
final_filtered_df = pd.concat(filtered_df_list, ignore_index=True)

final_filtered_df

#----------------------------
# Define the polygon for Field 2 as an ee.Geometry
aoi2 = ee.Geometry.Polygon([farm2_coordinates])

# Get the NDVI, RECI, and LSWI time series for Field 2
ndvi_timeseries_f2 = get_index_timeseries(aoi2, 'NDVI')
reci_timeseries_f2 = get_index_timeseries(aoi2, 'RECI')
lswi_timeseries_f2 = get_index_timeseries(aoi2, 'LSWI')

# Convert the time series to DataFrames for Field 2
ndvi_df_f2 = extract_index_data(ndvi_timeseries_f2, 'NDVI')
reci_df_f2 = extract_index_data(reci_timeseries_f2, 'RECI')
lswi_df_f2 = extract_index_data(lswi_timeseries_f2, 'LSWI')

# Merge the DataFrames on the Date for Field 2
merged_df_f2 = pd.merge(pd.merge(ndvi_df_f2, reci_df_f2, on='Date'), lswi_df_f2, on='Date')

# Create an empty DataFrame to accumulate the filtered results for Field 2
filtered_df_list_f2 = []

# Loop over the slicing periods and apply the filtering for Field 2
for i in range(len(slicing_periods) - 1):
    start_date = slicing_periods[i]
    end_date = slicing_periods[i + 1]

    # Apply filtering for each index within the specified date range for Field 2
    ndvi_filtered_f2 = filter_by_percentile_slicing(merged_df_f2, 'NDVI', start_date, end_date, percentile=0.75)
    reci_filtered_f2 = filter_by_percentile_slicing(merged_df_f2, 'RECI', start_date, end_date, percentile=0.75)
    lswi_filtered_f2 = filter_by_percentile_slicing(merged_df_f2, 'LSWI', start_date, end_date, percentile=0.75)

    # Merge the filtered results by Date for Field 2
    filtered_slice_f2 = pd.merge(pd.merge(ndvi_filtered_f2[['Date', 'NDVI']],
                                          reci_filtered_f2[['Date', 'RECI']], on='Date'),
                                 lswi_filtered_f2[['Date', 'LSWI']], on='Date')

    # Append the filtered slice to the list for Field 2
    filtered_df_list_f2.append(filtered_slice_f2)

# Concatenate all the filtered slices into a final DataFrame for Field 2
final_filtered_df_f2 = pd.concat(filtered_df_list_f2, ignore_index=True)

In [ ]:
merged_df_f2

,Date,NDVI,RECI,LSWI
0,2019-01-02,0.169588,0.201671,0.144269
1,2019-01-07,0.313582,0.497866,0.048900
2,2019-01-12,0.418490,0.688939,0.004167
3,2019-01-17,0.407438,0.707286,0.020912
4,2019-01-22,0.379133,0.599062,-0.022947
5,2019-01-27,0.355042,0.571563,-0.046251
6,2019-02-01,0.337481,0.516877,-0.045047
7,2019-02-11,0.145538,0.189287,0.053475
8,2019-02-16,0.120998,0.158026,-0.046418
9,2019-02-26,0.083889,0.079553,0.200484


In [ ]:
slicing_periods

DatetimeIndex(['2019-01-31', '2019-04-30', '2019-07-31', '2019-10-31',
               '2020-01-31', '2020-04-30', '2020-07-31', '2020-10-31',
               '2021-01-31', '2021-04-30', '2021-07-31', '2021-10-31',
               '2022-01-31', '2022-04-30', '2022-07-31', '2022-10-31',
               '2023-01-31', '2023-04-30', '2023-07-31', '2023-10-31'],
              dtype='datetime64[ns]', freq='3M')

In [ ]:
final_filtered_df_f2

,Date,NDVI,RECI,LSWI
0,2019-10-09,0.757569,1.893046,0.233354
1,2019-10-14,0.688732,1.638397,0.243357
2,2019-12-03,0.583994,1.296980,0.159165
3,2020-02-01,0.428087,0.843792,0.164001
4,2020-02-16,0.530348,1.124768,0.210354
5,2020-04-21,0.246635,0.370612,-0.076403
6,2020-09-23,0.714215,1.643329,0.235955
7,2020-10-28,0.706414,1.713553,0.241570
8,2020-11-07,0.695159,1.658145,0.249082
9,2020-12-02,0.665076,1.490586,0.247361


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig_scatter = go.Figure()

# Scatter Plot
# Add Final filtered - NDVI, RECI, and LSWI values
fig_scatter.add_trace(go.Scatter(x=final_filtered_df['Date'], y=final_filtered_df['NDVI'], mode='lines', name='NDVI (Filtered)',line=dict(dash='dot')))
fig_scatter.add_trace(go.Scatter(x=final_filtered_df['Date'], y=final_filtered_df['RECI'], mode='lines', name='RECI (Filtered)', line=dict(color='green',dash='dot')))
fig_scatter.add_trace(go.Scatter(x=final_filtered_df['Date'], y=final_filtered_df['LSWI'], mode='lines', name='LSWI (Filtered)', line=dict(color='blue',dash='dot')))

# Add original NDVI, RECI, and LSWI values
fig_scatter.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['NDVI'], mode='markers+lines', name='NDVI (Original)'))
fig_scatter.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['RECI'], mode='markers+lines', name='RECI (Original)', line=dict(color='green')))
fig_scatter.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['LSWI'], mode='markers+lines', name='LSWI (Original)', line=dict(color='blue')))

fig_scatter.update_layout(title=f'NDVI, RECI, and LSWI for 2019-2023 - Original vs Filtered', xaxis_title='Date', yaxis_title='Index Value',legend_title_text = 'Index')
fig_scatter.show()

# Box Plot
fig_box = go.Figure()

# Add NDVI box plot
fig_box.add_trace(go.Box(
    y=final_filtered_df['NDVI'],
    name='NDVI-filtered',
    marker_color='green'
))

fig_box.add_trace(go.Box(
    y=merged_df['NDVI'],
    name='NDVI-Original',
    marker_color='green'
))

# Add RECI box plot
fig_box.add_trace(go.Box(
    y=final_filtered_df['RECI'],
    name='RECI-filtered',
    marker_color='orange'
))

fig_box.add_trace(go.Box(
    y=merged_df['RECI'],
    name='RECI-Original',
    marker_color='orange'
))

# Add LSWI box plot
fig_box.add_trace(go.Box(
    y=final_filtered_df['LSWI'],
    name='LSWI-filtered',
    marker_color='blue'
))

fig_box.add_trace(go.Box(
    y=merged_df['LSWI'],
    name='LSWI-Original',
    marker_color='blue'
))

fig_box.update_layout(
    title='Box Plots of Filtered - NDVI, RECI, and LSWI',
    yaxis_title='Index Value',
    xaxis_title='Index',
    boxmode='group'  # Group boxes together
)

fig_box.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Farm 1 vs Farm 2 comparison graphs (Raw vs Filtered using 75<sup>th</sup> percentile)

In [ ]:
fig_scatter = go.Figure()

# Scatter Plot for Field 1 - Final filtered - NDVI, RECI, and LSWI values
fig_scatter.add_trace(go.Scatter(x=final_filtered_df['Date'], y=final_filtered_df['NDVI'], mode='lines', name='NDVI F1 (Filtered)', line=dict(dash='dot')))
fig_scatter.add_trace(go.Scatter(x=final_filtered_df['Date'], y=final_filtered_df['RECI'], mode='lines', name='RECI F1 (Filtered)', line=dict(color='green', dash='dot')))
fig_scatter.add_trace(go.Scatter(x=final_filtered_df['Date'], y=final_filtered_df['LSWI'], mode='lines', name='LSWI F1 (Filtered)', line=dict(color='blue', dash='dot')))

fig_scatter.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['NDVI'], mode='markers+lines', name='NDVI F1 (Original)'))
fig_scatter.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['RECI'], mode='markers+lines', name='RECI F1 (Original)', line=dict(color='green')))
fig_scatter.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df['LSWI'], mode='markers+lines', name='LSWI F1 (Original)', line=dict(color='blue')))

# Scatter Plot for Field 2 - Final filtered - NDVI, RECI, and LSWI values
fig_scatter.add_trace(go.Scatter(x=final_filtered_df_f2['Date'], y=final_filtered_df_f2['NDVI'], mode='lines', name='NDVI F2 (Filtered)', line=dict(dash='solid')))
fig_scatter.add_trace(go.Scatter(x=final_filtered_df_f2['Date'], y=final_filtered_df_f2['RECI'], mode='lines', name='RECI F2 (Filtered)', line=dict(color='green', dash='solid')))
fig_scatter.add_trace(go.Scatter(x=final_filtered_df_f2['Date'], y=final_filtered_df_f2['LSWI'], mode='lines', name='LSWI F2 (Filtered)', line=dict(color='blue', dash='solid')))

fig_scatter.add_trace(go.Scatter(x=merged_df_f2['Date'], y=merged_df_f2['NDVI'], mode='markers+lines', name='NDVI F2 (Original)'))
fig_scatter.add_trace(go.Scatter(x=merged_df_f2['Date'], y=merged_df_f2['RECI'], mode='markers+lines', name='RECI F2 (Original)', line=dict(color='red')))
fig_scatter.add_trace(go.Scatter(x=merged_df_f2['Date'], y=merged_df_f2['LSWI'], mode='markers+lines', name='LSWI F2 (Original)', line=dict(color='cyan')))

fig_scatter.update_layout(title=f'Comparison of NDVI, RECI, and LSWI for Field 1 and Field 2 (2019-2023)',
                          xaxis_title='Date',
                          yaxis_title='Index Value',
                          legend_title_text='Index')
fig_scatter.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Single year Raw graph vs Smoothed graphs

In [ ]:
# Ask the user to input the desired year
year = input("Enter the year for analysis (2019-2023): ")

# Convert the input year to an integer and filter the DataFrame
year = int(year)
single_year_df = merged_df[(merged_df['Date'] >= f'{year}-01-01') & (merged_df['Date'] < f'{year+1}-01-01')]

# Apply Savitzky-Golay filter
single_year_df['NDVI_SG'] = savgol_filter(single_year_df['NDVI'], window_length=7, polyorder=2)
single_year_df['RECI_SG'] = savgol_filter(single_year_df['RECI'], window_length=7, polyorder=2)
single_year_df['LSWI_SG'] = savgol_filter(single_year_df['LSWI'], window_length=7, polyorder=2)

# Apply Moving Average
single_year_df['NDVI_MA'] = single_year_df['NDVI'].rolling(window=5).mean()
single_year_df['RECI_MA'] = single_year_df['RECI'].rolling(window=5).mean()
single_year_df['LSWI_MA'] = single_year_df['LSWI'].rolling(window=5).mean()

# Apply Exponential Moving Average (EMA)
single_year_df['NDVI_EMA'] = single_year_df['NDVI'].ewm(span=5, adjust=False).mean()
single_year_df['RECI_EMA'] = single_year_df['RECI'].ewm(span=5, adjust=False).mean()
single_year_df['LSWI_EMA'] = single_year_df['LSWI'].ewm(span=5, adjust=False).mean()

# Apply Gaussian filter
single_year_df['NDVI_Gaussian'] = gaussian_filter1d(single_year_df['NDVI'], sigma=2)
single_year_df['RECI_Gaussian'] = gaussian_filter1d(single_year_df['RECI'], sigma=2)
single_year_df['LSWI_Gaussian'] = gaussian_filter1d(single_year_df['LSWI'], sigma=2)

# Apply LOESS (Locally Estimated Scatterplot Smoothing)
loess_ndvi = sm.nonparametric.lowess(single_year_df['NDVI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
loess_reci = sm.nonparametric.lowess(single_year_df['RECI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
loess_lswi = sm.nonparametric.lowess(single_year_df['LSWI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)

single_year_df['NDVI_LOESS'] = loess_ndvi[:, 1]
single_year_df['RECI_LOESS'] = loess_reci[:, 1]
single_year_df['LSWI_LOESS'] = loess_lswi[:, 1]

# Apply Kalman filter
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)

single_year_df['NDVI_Kalman'] = kf.em(single_year_df['NDVI'], n_iter=5).smooth(single_year_df['NDVI'])[0]
single_year_df['RECI_Kalman'] = kf.em(single_year_df['RECI'], n_iter=5).smooth(single_year_df['RECI'])[0]
single_year_df['LSWI_Kalman'] = kf.em(single_year_df['LSWI'], n_iter=5).smooth(single_year_df['LSWI'])[0]


Enter the year for analysis (2019-2023): 2019


In [ ]:
fig = go.Figure()

# Add original NDVI, RECI, and LSWI values
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI (Original)'))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['RECI'], mode='markers+lines', name='RECI (Original)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['LSWI'], mode='markers+lines', name='LSWI (Original)', line=dict(color='blue')))

# Add smoothed NDVI, RECI, and LSWI values for each smoothing technique
smoothing_methods = ['SG', 'MA', 'EMA', 'Gaussian', 'LOESS', 'Kalman']
colors = {'SG': 'orange', 'MA': 'purple', 'EMA': 'cyan', 'Gaussian': 'pink', 'LOESS': 'brown', 'Kalman': 'red'}

for method in smoothing_methods:
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'NDVI_{method}'], mode='lines', name=f'NDVI ({method})', line=dict(color=colors[method])))
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'RECI_{method}'], mode='lines', name=f'RECI ({method})', line=dict(color=colors[method], dash='dash')))
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'LSWI_{method}'], mode='lines', name=f'LSWI ({method})', line=dict(color=colors[method], dash='dot')))

fig.update_layout(title=f'NDVI, RECI, and LSWI for {year} with Various Smoothing Techniques', xaxis_title='Date', yaxis_title='Index Value',legend_title_text = 'Index')
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



# Two year comparison (A full season analysis)

In [ ]:
# Ask the user to input the desired start and end years
start_year = input("Enter the start year for analysis (2019-2023): ")
end_year = input("Enter the end year for analysis (2019-2023): ")

# Convert the input years to integers and filter the DataFrame
start_year = int(start_year)
end_year = int(end_year)
single_year_df = merged_df[(merged_df['Date'] >= f'{start_year}-01-01') & (merged_df['Date'] < f'{end_year+1}-01-01')]
single_year_farm2_df = merged_df_f2[(merged_df_f2['Date'] >= f'{start_year}-01-01') & (merged_df_f2['Date'] < f'{end_year+1}-01-01')]

# Apply Savitzky-Golay filter
single_year_df['NDVI_SG'] = savgol_filter(single_year_df['NDVI'], window_length=7, polyorder=3)
single_year_df['RECI_SG'] = savgol_filter(single_year_df['RECI'], window_length=7, polyorder=3)
single_year_df['LSWI_SG'] = savgol_filter(single_year_df['LSWI'], window_length=7, polyorder=3)

# Apply SG filter for farm2
single_year_farm2_df['NDVI_SG_f2'] = savgol_filter(single_year_farm2_df['NDVI'], window_length=7, polyorder=3)
single_year_farm2_df['RECI_SG_f2'] = savgol_filter(single_year_farm2_df['RECI'], window_length=7, polyorder=3)
single_year_farm2_df['LSWI_SG_f2'] = savgol_filter(single_year_farm2_df['LSWI'], window_length=7, polyorder=3)

# Apply Moving Average
single_year_df['NDVI_MA'] = single_year_df['NDVI'].rolling(window=5).mean()
single_year_df['RECI_MA'] = single_year_df['RECI'].rolling(window=5).mean()
single_year_df['LSWI_MA'] = single_year_df['LSWI'].rolling(window=5).mean()

# Apply MA for farm2
single_year_farm2_df['NDVI_MA_f2'] = single_year_farm2_df['NDVI'].rolling(window=5).mean()
single_year_farm2_df['reci_MA_f2'] = single_year_farm2_df['RECI'].rolling(window=5).mean()
single_year_farm2_df['lswi_MA_f2'] = single_year_farm2_df['LSWI'].rolling(window=5).mean()

# Apply Exponential Moving Average (EMA)
single_year_df['NDVI_EMA'] = single_year_df['NDVI'].ewm(span=5, adjust=False).mean()
single_year_df['RECI_EMA'] = single_year_df['RECI'].ewm(span=5, adjust=False).mean()
single_year_df['LSWI_EMA'] = single_year_df['LSWI'].ewm(span=5, adjust=False).mean()

# Apply EMA for farm2
single_year_farm2_df['NDVI_EMA_f2'] = single_year_farm2_df['NDVI'].ewm(span=5, adjust=False).mean()

# Apply Gaussian filter
single_year_df['NDVI_Gaussian'] = gaussian_filter1d(single_year_df['NDVI'], sigma=2)
single_year_df['RECI_Gaussian'] = gaussian_filter1d(single_year_df['RECI'], sigma=2)
single_year_df['LSWI_Gaussian'] = gaussian_filter1d(single_year_df['LSWI'], sigma=2)

# Apply Gaussian filter for farm2
single_year_farm2_df['NDVI_Gaussian_f2'] = gaussian_filter1d(single_year_farm2_df['NDVI'], sigma=2)

# Apply LOESS (Locally Estimated Scatterplot Smoothing)
loess_ndvi = sm.nonparametric.lowess(single_year_df['NDVI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
loess_reci = sm.nonparametric.lowess(single_year_df['RECI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
loess_lswi = sm.nonparametric.lowess(single_year_df['LSWI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)

loess_ndvi_f2 = sm.nonparametric.lowess(single_year_farm2_df['NDVI'], single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)

single_year_df['NDVI_LOESS'] = loess_ndvi[:, 1]
single_year_df['RECI_LOESS'] = loess_reci[:, 1]
single_year_df['LSWI_LOESS'] = loess_lswi[:, 1]

single_year_farm2_df['NDVI_LOESS_f2'] = loess_ndvi_f2[:, 1]

# Apply Kalman filter
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)

single_year_df['NDVI_Kalman'] = kf.em(single_year_df['NDVI'], n_iter=5).smooth(single_year_df['NDVI'])[0]
single_year_df['RECI_Kalman'] = kf.em(single_year_df['RECI'], n_iter=5).smooth(single_year_df['RECI'])[0]
single_year_df['LSWI_Kalman'] = kf.em(single_year_df['LSWI'], n_iter=5).smooth(single_year_df['LSWI'])[0]

single_year_farm2_df['NDVI_Kalman_f2'] = kf.em(single_year_farm2_df['NDVI'], n_iter=5).smooth(single_year_farm2_df['NDVI'])[0]

Enter the start year for analysis (2019-2023): 2019
Enter the end year for analysis (2019-2023): 2023


In [ ]:
single_year_farm2_df

,Date,NDVI,RECI,LSWI,NDVI_SG_f2,RECI_SG_f2,LSWI_SG_f2,NDVI_MA_f2,reci_MA_f2,lswi_MA_f2,NDVI_EMA_f2,NDVI_Gaussian_f2,NDVI_LOESS_f2,NDVI_Kalman_f2
0,2019-01-02,0.169588,0.201671,0.144269,0.165314,0.193209,0.138084,NaN,NaN,NaN,0.169588,0.290374,0.287348,0.250116
1,2019-01-07,0.313582,0.497866,0.048900,0.328650,0.524045,0.059780,NaN,NaN,NaN,0.217586,0.309557,0.282470,0.298743
2,2019-01-12,0.418490,0.688939,0.004167,0.401312,0.670600,0.014988,NaN,NaN,NaN,0.284554,0.334695,0.277771,0.341387
3,2019-01-17,0.407438,0.707286,0.020912,0.410930,0.688105,-0.007466,NaN,NaN,NaN,0.325515,0.349146,0.273270,0.352945
4,2019-01-22,0.379133,0.599062,-0.022947,0.408499,0.671889,-0.030479,0.337646,0.538965,0.039060,0.343388,0.342544,0.268981,0.342533
5,2019-01-27,0.355042,0.571563,-0.046251,0.350711,0.556779,-0.020193,0.374737,0.612943,0.000956,0.347273,0.313795,0.264910,0.317364
6,2019-02-01,0.337481,0.516877,-0.045047,0.280171,0.422896,-0.043946,0.379517,0.616745,-0.017833,0.344009,0.269760,0.261055,0.277005
7,2019-02-11,0.145538,0.189287,0.053475,0.188066,0.264274,0.023616,0.324926,0.516815,-0.007972,0.277852,0.222905,0.253959,0.212264
8,2019-02-16,0.120998,0.158026,-0.046418,0.126805,0.161610,0.057142,0.267638,0.406963,-0.021438,0.225567,0.186612,0.250687,0.174424
9,2019-02-26,0.083889,0.079553,0.200484,0.118780,0.148342,0.030316,0.208590,0.303061,0.023248,0.178341,0.168011,0.244616,0.158125


In [ ]:
fig = go.Figure()

# Add original NDVI, RECI, and LSWI values
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI F1 (Original)'))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['RECI'], mode='markers+lines', name='RECI F1 (Original)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['LSWI'], mode='markers+lines', name='LSWI F1 (Original)', line=dict(color='blue')))

# farm2
fig.add_trace(go.Scatter(x=single_year_farm2_df['Date'], y=single_year_farm2_df['NDVI'], mode='markers+lines', name='NDVI F2 (Original)'))

# Add smoothed NDVI, RECI, and LSWI values for each smoothing technique
smoothing_methods = ['SG', 'MA', 'EMA', 'Gaussian', 'LOESS', 'Kalman']
colors = {'SG': 'orange', 'MA': 'purple', 'EMA': 'cyan', 'Gaussian': 'pink', 'LOESS': 'brown', 'Kalman': 'red'}

for method in smoothing_methods:
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'NDVI_{method}'], mode='lines', name=f'NDVI F1 ({method})', line=dict(color=colors[method])))
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'RECI_{method}'], mode='lines', name=f'RECI F1 ({method})', line=dict(color=colors[method], dash='dash')))
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'LSWI_{method}'], mode='lines', name=f'LSWI F1 ({method})', line=dict(color=colors[method], dash='dot')))

for method in smoothing_methods:
    fig.add_trace(go.Scatter(x=single_year_farm2_df['Date'], y=single_year_farm2_df[f'NDVI_{method}_f2'], mode='lines', name=f'NDVI F2 ({method})', line=dict(color=colors[method])))

# Define the year dynamically from the data or user input
year = single_year_df['Date'].dt.year.iloc[0]  # Assuming all dates are in the same year, or use a specific year

# Define the specific periods for shading
wet_season_start = f'{year}-04-01'
wet_season_end = f'{year}-10-31'

rainfall_peak_start = f'{year}-08-01'
rainfall_peak_end = f'{year}-09-30'

dry_season_start = f'{year}-11-01'
dry_season_end = f'{year+1}-03-31'  # Extends into the next year

# Add shaded region for wet season (April - October)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=wet_season_start,
    y0=0,
    x1=wet_season_end,
    y1=1,
    fillcolor="LightGreen",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for rainfall peak (August - September)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=rainfall_peak_start,
    y0=0,
    x1=rainfall_peak_end,
    y1=1,
    fillcolor="LightCoral",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for dry season (November - March)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=dry_season_start,
    y0=0,
    x1=dry_season_end,
    y1=1,
    fillcolor="Yellow",
    opacity=0.2,
    layer="below",
    line_width=0,
)

# Update layout with title and labels
fig.update_layout(
    title=f'NDVI, RECI, and LSWI for {year}-{year+1} with Various Smoothing Techniques',
    xaxis_title='Date',
    yaxis_title='Index Value',
    legend_title_text='Index'
)

# Show the figure
fig.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Spline vs wavelet smoothing

In [ ]:
import numpy as np
from scipy.interpolate import UnivariateSpline
from scipy.fft import fft, ifft
import pywt

# Assuming single_year_df is already defined

# Spline Smoothing
spline_ndvi = UnivariateSpline(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['NDVI'], s=0.9)
spline_reci = UnivariateSpline(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['RECI'], s=1)
spline_lswi = UnivariateSpline(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['LSWI'], s=1)

single_year_df['NDVI_Spline'] = spline_ndvi(single_year_df['Date'].map(pd.Timestamp.toordinal))
single_year_df['RECI_Spline'] = spline_reci(single_year_df['Date'].map(pd.Timestamp.toordinal))
single_year_df['LSWI_Spline'] = spline_lswi(single_year_df['Date'].map(pd.Timestamp.toordinal))

# Wavelet Smoothing
def wavelet_smoothing(signal):
    coeffs = pywt.wavedec(signal, 'db1', level=6)
    coeffs[1:] = [pywt.threshold(c, value=0.5, mode='soft') for c in coeffs[1:]]
    smoothed_signal = pywt.waverec(coeffs, 'db1')
    # return pywt.waverec(coeffs, 'db1')
    # Trim the smoothed signal to match the original length if necessary
    return smoothed_signal[:len(signal)] # Added this line to fix the length mismatch

single_year_df['NDVI_Wavelet'] = wavelet_smoothing(single_year_df['NDVI'])
single_year_df['RECI_Wavelet'] = wavelet_smoothing(single_year_df['RECI'])
single_year_df['LSWI_Wavelet'] = wavelet_smoothing(single_year_df['LSWI'])

# Fourier Transform Smoothing
# def fourier_smoothing(signal, threshold=0.1):
#     # Convert the signal to a NumPy array and ensure it's aligned
#     signal_array = np.asarray(signal)  # Convert to NumPy array if it's not already
#     signal_array = np.require(signal_array, dtype=np.float64, requirements=['ALIGNED'])  # Ensure alignment
#     fft_signal = fft(signal)
#     fft_signal[np.abs(fft_signal) < threshold * np.max(np.abs(fft_signal))] = 0
#     return np.real(ifft(fft_signal))

# single_year_df['NDVI_Fourier'] = fourier_smoothing(single_year_df['NDVI'])
# single_year_df['RECI_Fourier'] = fourier_smoothing(single_year_df['RECI'])
# single_year_df['LSWI_Fourier'] = fourier_smoothing(single_year_df['LSWI'])

# Visualization
fig = go.Figure()

# Add original NDVI, RECI, and LSWI values
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI (Original)'))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['RECI'], mode='markers+lines', name='RECI (Original)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['LSWI'], mode='markers+lines', name='LSWI (Original)', line=dict(color='blue')))

# Add smoothed NDVI, RECI, and LSWI values for each smoothing technique
smoothing_methods = ['SG', 'MA', 'EMA', 'Gaussian', 'LOESS', 'Kalman', 'Spline', 'Wavelet'] # 'Fourier'
colors = {'SG': 'orange', 'MA': 'purple', 'EMA': 'cyan', 'Gaussian': 'pink', 'LOESS': 'brown', 'Kalman': 'red', 'Spline': 'magenta', 'Wavelet': 'teal'} #  'Fourier': 'gold'

for method in smoothing_methods:
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'NDVI_{method}'], mode='lines', name=f'NDVI ({method})', line=dict(color=colors[method])))
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'RECI_{method}'], mode='lines', name=f'RECI ({method})', line=dict(color=colors[method], dash='dash')))
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'LSWI_{method}'], mode='lines', name=f'LSWI ({method})', line=dict(color=colors[method], dash='dot')))

# Define the year dynamically from the data or user input
year = single_year_df['Date'].dt.year.iloc[0]  # Assuming all dates are in the same year, or use a specific year

# Define the specific periods for shading
wet_season_start = f'{year}-04-01'
wet_season_end = f'{year}-10-31'

rainfall_peak_start = f'{year}-08-01'
rainfall_peak_end = f'{year}-09-30'

dry_season_start = f'{year}-11-01'
dry_season_end = f'{year+1}-03-31'  # Extends into the next year

# Add shaded region for wet season (April - October)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=wet_season_start,
    y0=0,
    x1=wet_season_end,
    y1=1,
    fillcolor="LightGreen",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for rainfall peak (August - September)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=rainfall_peak_start,
    y0=0,
    x1=rainfall_peak_end,
    y1=1,
    fillcolor="LightCoral",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for dry season (November - March)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=dry_season_start,
    y0=0,
    x1=dry_season_end,
    y1=1,
    fillcolor="Yellow",
    opacity=0.2,
    layer="below",
    line_width=0,
)

# Update layout with title and labels
fig.update_layout(
    title=f'NDVI, RECI, and LSWI for {year}-{year+1} with Various Smoothing Techniques',
    xaxis_title='Date',
    yaxis_title='Index Value',
    legend_title_text='Index'
)

# Show the figure
fig.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Univariate Spline and BSpline Smoothening applied to NDVI only

In [ ]:
# Spline Smoothing (Univariate Spline)
spline_ndvi = UnivariateSpline(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['NDVI'], s=0.5)
spline_ndvi_f2 = UnivariateSpline(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), single_year_farm2_df['NDVI'], s=0.5)

single_year_df['NDVI_Spline'] = spline_ndvi(single_year_df['Date'].map(pd.Timestamp.toordinal))
single_year_farm2_df['NDVI_Spline_f2'] = spline_ndvi_f2(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal))

# BSpline Smoothing
tck = splrep(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['NDVI'], s=0.65)
bspline_ndvi = splev(single_year_df['Date'].map(pd.Timestamp.toordinal), tck)
single_year_df['NDVI_BSpline'] = bspline_ndvi

tck_f2 = splrep(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), single_year_farm2_df['NDVI'], s=0.65)
bspline_ndvi_f2 = splev(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), tck_f2)
single_year_farm2_df['NDVI_BSpline_f2'] = bspline_ndvi_f2

# Wavelet Smoothing
def wavelet_smoothing(signal):
    coeffs = pywt.wavedec(signal, 'db1', level=6)
    coeffs[1:] = [pywt.threshold(c, value=0.5, mode='soft') for c in coeffs[1:]]
    smoothed_signal = pywt.waverec(coeffs, 'db1')
    return smoothed_signal[:len(signal)]  # Trim to match original length

single_year_df['NDVI_Wavelet'] = wavelet_smoothing(single_year_df['NDVI'])
single_year_farm2_df['NDVI_Wavelet_f2'] = wavelet_smoothing(single_year_farm2_df['NDVI'])

# Fourier Transform Smoothing (Optional, commented out)
# def fourier_smoothing(signal, threshold=0.1):
#     fft_signal = fft(signal)
#     fft_signal[np.abs(fft_signal) < threshold * np.max(np.abs(fft_signal))] = 0
#     return np.real(ifft(fft_signal))

# single_year_df['NDVI_Fourier'] = fourier_smoothing(single_year_df['NDVI'])

# Visualization
fig = go.Figure()

# Add original NDVI values
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI F1 (Original)'))
fig.add_trace(go.Scatter(x=single_year_farm2_df['Date'], y=single_year_farm2_df['NDVI'], mode='markers+lines', name='NDVI F2 (Original)'))

# Add smoothed NDVI values for each smoothing technique
smoothing_methods = ['SG', 'MA', 'EMA', 'Gaussian', 'LOESS', 'Kalman', 'Spline', 'Wavelet', 'BSpline']  # 'Fourier' if using Fourier
colors = {'SG': 'orange', 'MA': 'purple', 'EMA': 'cyan', 'Gaussian': 'pink', 'LOESS': 'brown', 'Kalman': 'red', 'Spline': 'magenta', 'Wavelet': 'teal', 'BSpline': 'blue'}  # Add color for Fourier if used

for method in smoothing_methods:
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'NDVI_{method}'], mode='lines', name=f'NDVI F1 ({method})', line=dict(color=colors[method])))
    fig.add_trace(go.Scatter(x=single_year_farm2_df['Date'], y=single_year_farm2_df[f'NDVI_{method}_f2'], mode='lines', name=f'NDVI F2 ({method})', line=dict(color=colors[method])))

# Define the year dynamically from the data or user input
year = single_year_df['Date'].dt.year.iloc[0]  # Assuming all dates are in the same year, or use a specific year

# Define the specific periods for shading
wet_season_start = f'{year}-04-01'
wet_season_end = f'{year}-10-31'

rainfall_peak_start = f'{year}-08-01'
rainfall_peak_end = f'{year}-09-30'

dry_season_start = f'{year}-11-01'
dry_season_end = f'{year+1}-03-31'  # Extends into the next year

# Add shaded region for wet season (April - October)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=wet_season_start,
    y0=0,
    x1=wet_season_end,
    y1=1,
    fillcolor="LightGreen",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for rainfall peak (August - September)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=rainfall_peak_start,
    y0=0,
    x1=rainfall_peak_end,
    y1=1,
    fillcolor="LightCoral",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for dry season (November - March)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=dry_season_start,
    y0=0,
    x1=dry_season_end,
    y1=1,
    fillcolor="Yellow",
    opacity=0.2,
    layer="below",
    line_width=0,
)

# Update layout with title and labels
fig.update_layout(
    title=f'NDVI for {year}-{year+1} with Various Smoothing Techniques',
    xaxis_title='Date',
    yaxis_title='NDVI Value',
    legend_title_text='NDVI Smoothing Methods'
)

# Show the figure
fig.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Dropping NDVI values < 0.15

In [4]:
# Define the coordinates for the AOI (single polygon) and second farm
coordinates = [(11.9339692, 9.6092636), (11.9378834, 9.607271), (11.937092, 9.590627), (11.9297, 9.591299)]  # Dangote sugarcane field co-ordinates in Nigeria
farm2_coordinates = [(11.876698, 9.611602), (11.885893, 9.611618), (11.885875, 9.600578), (11.876751, 9.600584)]  # Dangote sugarcane field2

# Define the time range
start_date = '2019-01-01'
end_date = '2023-12-31'

# Function to mask clouds based on the QA60 band of Sentinel-2
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(cloud_mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Load Sentinel-2 data, filter by date and bounds, apply cloud mask, and calculate NDVI
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .map(mask_clouds) \
    .map(calculate_ndvi)

# Define the polygons as ee.Geometry for both farms
aoi = ee.Geometry.Polygon([coordinates])
aoi2 = ee.Geometry.Polygon([farm2_coordinates])

# Function to calculate mean NDVI for the polygon over time
def get_ndvi_timeseries(aoi):
    def compute_ndvi(image):
        mean_ndvi = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=30
        ).get('NDVI')
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'date': date, 'NDVI': mean_ndvi})

    ndvi_collection = collection.filterBounds(aoi).select('NDVI').map(compute_ndvi)
    return ndvi_collection.getInfo()

# Get the NDVI time series for both farms
ndvi_timeseries_farm1 = get_ndvi_timeseries(aoi)
ndvi_timeseries_farm2 = get_ndvi_timeseries(aoi2)

# Convert the NDVI time series to DataFrames for both farms
def extract_ndvi_data(ndvi_timeseries):
    dates = []
    ndvi_values = []

    for feature in ndvi_timeseries['features']:
        properties = feature['properties']
        date = properties.get('date')
        ndvi = properties.get('NDVI')
        if date and ndvi is not None and ndvi >= 0.15:  # Filter out NDVI values lower than 0.15
            dates.append(date)
            ndvi_values.append(ndvi)

    return pd.DataFrame({'Date': pd.to_datetime(dates), 'NDVI': ndvi_values})

# Create DataFrames for NDVI for both farms
ndvi_df_farm1 = extract_ndvi_data(ndvi_timeseries_farm1)
ndvi_df_farm2 = extract_ndvi_data(ndvi_timeseries_farm2)

# # Function to filter the DataFrame based on the 75th percentile within each slice
# def filter_by_percentile_slicing(df, start_date, end_date, percentile=0.75):
#     # Slice the DataFrame based on the date range
#     df_slice = df[(df['Date'] >= start_date) & (df['Date'] < end_date)]

#     # Calculate the 75th percentile for the NDVI in this slice
#     upper_bound = df_slice['NDVI'].quantile(percentile)

#     # Filter the slice to keep only the values above or equal to the 75th percentile
#     return df_slice[df_slice['NDVI'] >= upper_bound]

# Define the slicing periods (example: every 3 months)
# slicing_periods = pd.date_range(start='2019-01-01', end='2023-12-31', freq='3M')

# # Create empty DataFrames to accumulate the filtered results for both farms
# filtered_df_list_farm1 = []
# filtered_df_list_farm2 = []

# # Loop over the slicing periods and apply the filtering for both farms
# for i in range(len(slicing_periods) - 1):
#     start_date = slicing_periods[i]
#     end_date = slicing_periods[i + 1]

#     # Apply filtering for NDVI within the specified date range for both farms
#     ndvi_filtered_farm1 = filter_by_percentile_slicing(ndvi_df_farm1, start_date, end_date, percentile=0.75)
#     ndvi_filtered_farm2 = filter_by_percentile_slicing(ndvi_df_farm2, start_date, end_date, percentile=0.75)

#     # Append the filtered slices to the respective lists
#     filtered_df_list_farm1.append(ndvi_filtered_farm1)
#     filtered_df_list_farm2.append(ndvi_filtered_farm2)

# # Concatenate all the filtered slices into final DataFrames for both farms
# final_filtered_df_farm1 = pd.concat(filtered_df_list_farm1, ignore_index=True)
# final_filtered_df_farm2 = pd.concat(filtered_df_list_farm2, ignore_index=True)

# The final_filtered_df_farm1 and final_filtered_df_farm2 DataFrames now contain the filtered NDVI values
# for both fields after removing values lower than 0.15 and keeping only those in the 75th percentile.


In [5]:
# Ask the user to input the desired start and end years
start_year = input("Enter the start year for analysis (2019-2023): ")
end_year = input("Enter the end year for analysis (2019-2023): ")

# Convert the input years to integers and filter the DataFrame
start_year = int(start_year)
end_year = int(end_year)
single_year_df = ndvi_df_farm1[(ndvi_df_farm1['Date'] >= f'{start_year}-01-01') & (ndvi_df_farm1['Date'] < f'{end_year+1}-01-01')]
single_year_farm2_df = ndvi_df_farm2[(ndvi_df_farm2['Date'] >= f'{start_year}-01-01') & (ndvi_df_farm2['Date'] < f'{end_year+1}-01-01')]

# Apply Savitzky-Golay filter
single_year_df['NDVI_SG'] = savgol_filter(single_year_df['NDVI'], window_length=7, polyorder=3)
# single_year_df['RECI_SG'] = savgol_filter(single_year_df['RECI'], window_length=7, polyorder=3)
# single_year_df['LSWI_SG'] = savgol_filter(single_year_df['LSWI'], window_length=7, polyorder=3)

# Apply SG filter for farm2
single_year_farm2_df['NDVI_SG_f2'] = savgol_filter(single_year_farm2_df['NDVI'], window_length=7, polyorder=3)
# single_year_farm2_df['RECI_SG_f2'] = savgol_filter(single_year_farm2_df['RECI'], window_length=7, polyorder=3)
# single_year_farm2_df['LSWI_SG_f2'] = savgol_filter(single_year_farm2_df['LSWI'], window_length=7, polyorder=3)

# Apply Moving Average
single_year_df['NDVI_MA'] = single_year_df['NDVI'].rolling(window=5).mean()
# single_year_df['RECI_MA'] = single_year_df['RECI'].rolling(window=5).mean()
# single_year_df['LSWI_MA'] = single_year_df['LSWI'].rolling(window=5).mean()

# Apply MA for farm2
single_year_farm2_df['NDVI_MA_f2'] = single_year_farm2_df['NDVI'].rolling(window=5).mean()
# single_year_farm2_df['reci_MA_f2'] = single_year_farm2_df['RECI'].rolling(window=5).mean()
# single_year_farm2_df['lswi_MA_f2'] = single_year_farm2_df['LSWI'].rolling(window=5).mean()

# Apply Exponential Moving Average (EMA)
single_year_df['NDVI_EMA'] = single_year_df['NDVI'].ewm(span=5, adjust=False).mean()
# single_year_df['RECI_EMA'] = single_year_df['RECI'].ewm(span=5, adjust=False).mean()
# single_year_df['LSWI_EMA'] = single_year_df['LSWI'].ewm(span=5, adjust=False).mean()

# Apply EMA for farm2
single_year_farm2_df['NDVI_EMA_f2'] = single_year_farm2_df['NDVI'].ewm(span=5, adjust=False).mean()

# Apply Gaussian filter
single_year_df['NDVI_Gaussian'] = gaussian_filter1d(single_year_df['NDVI'], sigma=2)
# single_year_df['RECI_Gaussian'] = gaussian_filter1d(single_year_df['RECI'], sigma=2)
# single_year_df['LSWI_Gaussian'] = gaussian_filter1d(single_year_df['LSWI'], sigma=2)

# Apply Gaussian filter for farm2
single_year_farm2_df['NDVI_Gaussian_f2'] = gaussian_filter1d(single_year_farm2_df['NDVI'], sigma=2)

# Apply LOESS (Locally Estimated Scatterplot Smoothing)
loess_ndvi = sm.nonparametric.lowess(single_year_df['NDVI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
# loess_reci = sm.nonparametric.lowess(single_year_df['RECI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
# loess_lswi = sm.nonparametric.lowess(single_year_df['LSWI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)

loess_ndvi_f2 = sm.nonparametric.lowess(single_year_farm2_df['NDVI'], single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)

single_year_df['NDVI_LOESS'] = loess_ndvi[:, 1]
# single_year_df['RECI_LOESS'] = loess_reci[:, 1]
# single_year_df['LSWI_LOESS'] = loess_lswi[:, 1]

single_year_farm2_df['NDVI_LOESS_f2'] = loess_ndvi_f2[:, 1]

# Apply Kalman filter
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)

single_year_df['NDVI_Kalman'] = kf.em(single_year_df['NDVI'], n_iter=5).smooth(single_year_df['NDVI'])[0]
# single_year_df['RECI_Kalman'] = kf.em(single_year_df['RECI'], n_iter=5).smooth(single_year_df['RECI'])[0]
# single_year_df['LSWI_Kalman'] = kf.em(single_year_df['LSWI'], n_iter=5).smooth(single_year_df['LSWI'])[0]

single_year_farm2_df['NDVI_Kalman_f2'] = kf.em(single_year_farm2_df['NDVI'], n_iter=5).smooth(single_year_farm2_df['NDVI'])[0]

Enter the start year for analysis (2019-2023): 2019
Enter the end year for analysis (2019-2023): 2023


In [6]:
# Spline Smoothing (Univariate Spline)
spline_ndvi = UnivariateSpline(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['NDVI'], s=0.5)
spline_ndvi_f2 = UnivariateSpline(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), single_year_farm2_df['NDVI'], s=0.5)

single_year_df['NDVI_Spline'] = spline_ndvi(single_year_df['Date'].map(pd.Timestamp.toordinal))
single_year_farm2_df['NDVI_Spline_f2'] = spline_ndvi_f2(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal))

# BSpline Smoothing
tck = splrep(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['NDVI'], s=0.65)
bspline_ndvi = splev(single_year_df['Date'].map(pd.Timestamp.toordinal), tck)
single_year_df['NDVI_BSpline'] = bspline_ndvi

tck_f2 = splrep(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), single_year_farm2_df['NDVI'], s=0.65)
bspline_ndvi_f2 = splev(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), tck_f2)
single_year_farm2_df['NDVI_BSpline_f2'] = bspline_ndvi_f2

# Wavelet Smoothing
def wavelet_smoothing(signal):
    coeffs = pywt.wavedec(signal, 'db1', level=6)
    coeffs[1:] = [pywt.threshold(c, value=0.5, mode='soft') for c in coeffs[1:]]
    smoothed_signal = pywt.waverec(coeffs, 'db1')
    return smoothed_signal[:len(signal)]  # Trim to match original length

single_year_df['NDVI_Wavelet'] = wavelet_smoothing(single_year_df['NDVI'])
single_year_farm2_df['NDVI_Wavelet_f2'] = wavelet_smoothing(single_year_farm2_df['NDVI'])

# Fourier Transform Smoothing (Optional, commented out)
# def fourier_smoothing(signal, threshold=0.1):
#     fft_signal = fft(signal)
#     fft_signal[np.abs(fft_signal) < threshold * np.max(np.abs(fft_signal))] = 0
#     return np.real(ifft(fft_signal))

# single_year_df['NDVI_Fourier'] = fourier_smoothing(single_year_df['NDVI'])

# Visualization
fig = go.Figure()

# Add original NDVI values
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI F1 (Original)'))
fig.add_trace(go.Scatter(x=single_year_farm2_df['Date'], y=single_year_farm2_df['NDVI'], mode='markers+lines', name='NDVI F2 (Original)'))

# Add smoothed NDVI values for each smoothing technique
smoothing_methods = ['SG', 'MA', 'EMA', 'Gaussian', 'LOESS', 'Kalman', 'Spline', 'Wavelet', 'BSpline']  # 'Fourier' if using Fourier
colors = {'SG': 'orange', 'MA': 'purple', 'EMA': 'cyan', 'Gaussian': 'pink', 'LOESS': 'brown', 'Kalman': 'red', 'Spline': 'magenta', 'Wavelet': 'teal', 'BSpline': 'blue'}  # Add color for Fourier if used

for method in smoothing_methods:
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'NDVI_{method}'], mode='lines', name=f'NDVI F1 ({method})', line=dict(color=colors[method])))
    fig.add_trace(go.Scatter(x=single_year_farm2_df['Date'], y=single_year_farm2_df[f'NDVI_{method}_f2'], mode='lines', name=f'NDVI F2 ({method})', line=dict(color=colors[method])))

# Define the year dynamically from the data or user input
year = single_year_df['Date'].dt.year.iloc[0]  # Assuming all dates are in the same year, or use a specific year

# Define the specific periods for shading
wet_season_start = f'{year}-04-01'
wet_season_end = f'{year}-10-31'

rainfall_peak_start = f'{year}-08-01'
rainfall_peak_end = f'{year}-09-30'

dry_season_start = f'{year}-11-01'
dry_season_end = f'{year+1}-03-31'  # Extends into the next year

# Add shaded region for wet season (April - October)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=wet_season_start,
    y0=0,
    x1=wet_season_end,
    y1=1,
    fillcolor="LightGreen",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for rainfall peak (August - September)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=rainfall_peak_start,
    y0=0,
    x1=rainfall_peak_end,
    y1=1,
    fillcolor="LightCoral",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for dry season (November - March)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=dry_season_start,
    y0=0,
    x1=dry_season_end,
    y1=1,
    fillcolor="Yellow",
    opacity=0.2,
    layer="below",
    line_width=0,
)

# Update layout with title and labels
fig.update_layout(
    title=f'NDVI for {year}-{year+1} with Various Smoothing Techniques which the values <0.15 are dropped',
    xaxis_title='Date',
    yaxis_title='NDVI Value',
    legend_title_text='NDVI Smoothing Methods'
)

# Show the figure
fig.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Filter NDVI <0.15 and peaks only

In [ ]:
import ee
import pandas as pd

# Define the coordinates for the AOI (single polygon) and second farm
coordinates = [(11.9339692, 9.6092636), (11.9378834, 9.607271), (11.937092, 9.590627), (11.9297, 9.591299)]  # Dangote sugarcane field co-ordinates in Nigeria
farm2_coordinates = [(11.876698, 9.611602), (11.885893, 9.611618), (11.885875, 9.600578), (11.876751, 9.600584)]  # Dangote sugarcane field2

# Define the time range
start_date = '2019-01-01'
end_date = '2023-12-31'

# Function to mask clouds based on the QA60 band of Sentinel-2
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(cloud_mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Load Sentinel-2 data, filter by date and bounds, apply cloud mask, and calculate NDVI
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .map(mask_clouds) \
    .map(calculate_ndvi)

# Define the polygons as ee.Geometry for both farms
aoi = ee.Geometry.Polygon([coordinates])
aoi2 = ee.Geometry.Polygon([farm2_coordinates])

# Function to calculate mean NDVI for the polygon over time
def get_ndvi_timeseries(aoi):
    def compute_ndvi(image):
        mean_ndvi = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=30
        ).get('NDVI')
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'date': date, 'NDVI': mean_ndvi})

    ndvi_collection = collection.filterBounds(aoi).select('NDVI').map(compute_ndvi)
    return ndvi_collection.getInfo()

# Get the NDVI time series for both farms
ndvi_timeseries_farm1 = get_ndvi_timeseries(aoi)
ndvi_timeseries_farm2 = get_ndvi_timeseries(aoi2)

# Convert the NDVI time series to DataFrames for both farms
def extract_ndvi_data(ndvi_timeseries):
    dates = []
    ndvi_values = []

    for feature in ndvi_timeseries['features']:
        properties = feature['properties']
        date = properties.get('date')
        ndvi = properties.get('NDVI')
        if date and ndvi is not None and ndvi >= 0.15:  # Filter out NDVI values lower than 0.15
            dates.append(date)
            ndvi_values.append(ndvi)

    return pd.DataFrame({'Date': pd.to_datetime(dates), 'NDVI': ndvi_values})

# Create DataFrames for NDVI for both farms
ndvi_df_farm1 = extract_ndvi_data(ndvi_timeseries_farm1)
ndvi_df_farm2 = extract_ndvi_data(ndvi_timeseries_farm2)

# Custom function to detect peaks in the NDVI data
def detect_peaks(ndvi_df):
    peaks = []
    for i in range(1, len(ndvi_df) - 1):
        if ndvi_df['NDVI'][i] > ndvi_df['NDVI'][i - 1] and ndvi_df['NDVI'][i] > ndvi_df['NDVI'][i + 1]:
            peaks.append((ndvi_df['Date'][i], ndvi_df['NDVI'][i]))

    # Include the first and last points if they are higher than their single neighbor
    if ndvi_df['NDVI'][0] > ndvi_df['NDVI'][1]:
        peaks.insert(0, (ndvi_df['Date'][0], ndvi_df['NDVI'][0]))
    if ndvi_df['NDVI'][len(ndvi_df) - 1] > ndvi_df['NDVI'][len(ndvi_df) - 2]:
        peaks.append((ndvi_df['Date'][len(ndvi_df) - 1], ndvi_df['NDVI'][len(ndvi_df) - 1]))

    peak_dates, peak_values = zip(*peaks)
    return pd.DataFrame({'Date': pd.to_datetime(peak_dates), 'NDVI': peak_values})

# Detect peaks for both farms
peak_ndvi_df_farm1 = detect_peaks(ndvi_df_farm1)
peak_ndvi_df_farm2 = detect_peaks(ndvi_df_farm2)

In [ ]:
# Ask the user to input the desired start and end years
start_year = input("Enter the start year for analysis (2019-2023): ")
end_year = input("Enter the end year for analysis (2019-2023): ")

# Convert the input years to integers and filter the DataFrame
start_year = int(start_year)
end_year = int(end_year)
single_year_df = peak_ndvi_df_farm1[(peak_ndvi_df_farm1['Date'] >= f'{start_year}-01-01') & (peak_ndvi_df_farm1['Date'] < f'{end_year+1}-01-01')]
single_year_farm2_df = peak_ndvi_df_farm2[(peak_ndvi_df_farm2['Date'] >= f'{start_year}-01-01') & (peak_ndvi_df_farm2['Date'] < f'{end_year+1}-01-01')]

# Apply Savitzky-Golay filter
single_year_df['NDVI_SG'] = savgol_filter(single_year_df['NDVI'], window_length=7, polyorder=3)
# single_year_df['RECI_SG'] = savgol_filter(single_year_df['RECI'], window_length=7, polyorder=3)
# single_year_df['LSWI_SG'] = savgol_filter(single_year_df['LSWI'], window_length=7, polyorder=3)

# Apply SG filter for farm2
single_year_farm2_df['NDVI_SG_f2'] = savgol_filter(single_year_farm2_df['NDVI'], window_length=7, polyorder=3)
# single_year_farm2_df['RECI_SG_f2'] = savgol_filter(single_year_farm2_df['RECI'], window_length=7, polyorder=3)
# single_year_farm2_df['LSWI_SG_f2'] = savgol_filter(single_year_farm2_df['LSWI'], window_length=7, polyorder=3)

# Apply Moving Average
single_year_df['NDVI_MA'] = single_year_df['NDVI'].rolling(window=5).mean()
# single_year_df['RECI_MA'] = single_year_df['RECI'].rolling(window=5).mean()
# single_year_df['LSWI_MA'] = single_year_df['LSWI'].rolling(window=5).mean()

# Apply MA for farm2
single_year_farm2_df['NDVI_MA_f2'] = single_year_farm2_df['NDVI'].rolling(window=5).mean()
# single_year_farm2_df['reci_MA_f2'] = single_year_farm2_df['RECI'].rolling(window=5).mean()
# single_year_farm2_df['lswi_MA_f2'] = single_year_farm2_df['LSWI'].rolling(window=5).mean()

# Apply Exponential Moving Average (EMA)
single_year_df['NDVI_EMA'] = single_year_df['NDVI'].ewm(span=5, adjust=False).mean()
# single_year_df['RECI_EMA'] = single_year_df['RECI'].ewm(span=5, adjust=False).mean()
# single_year_df['LSWI_EMA'] = single_year_df['LSWI'].ewm(span=5, adjust=False).mean()

# Apply EMA for farm2
single_year_farm2_df['NDVI_EMA_f2'] = single_year_farm2_df['NDVI'].ewm(span=5, adjust=False).mean()

# Apply Gaussian filter
single_year_df['NDVI_Gaussian'] = gaussian_filter1d(single_year_df['NDVI'], sigma=2)
# single_year_df['RECI_Gaussian'] = gaussian_filter1d(single_year_df['RECI'], sigma=2)
# single_year_df['LSWI_Gaussian'] = gaussian_filter1d(single_year_df['LSWI'], sigma=2)

# Apply Gaussian filter for farm2
single_year_farm2_df['NDVI_Gaussian_f2'] = gaussian_filter1d(single_year_farm2_df['NDVI'], sigma=2)

# Apply LOESS (Locally Estimated Scatterplot Smoothing)
loess_ndvi = sm.nonparametric.lowess(single_year_df['NDVI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
# loess_reci = sm.nonparametric.lowess(single_year_df['RECI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)
# loess_lswi = sm.nonparametric.lowess(single_year_df['LSWI'], single_year_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)

loess_ndvi_f2 = sm.nonparametric.lowess(single_year_farm2_df['NDVI'], single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), frac=0.1)

single_year_df['NDVI_LOESS'] = loess_ndvi[:, 1]
# single_year_df['RECI_LOESS'] = loess_reci[:, 1]
# single_year_df['LSWI_LOESS'] = loess_lswi[:, 1]

single_year_farm2_df['NDVI_LOESS_f2'] = loess_ndvi_f2[:, 1]

# Apply Kalman filter
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)

single_year_df['NDVI_Kalman'] = kf.em(single_year_df['NDVI'], n_iter=5).smooth(single_year_df['NDVI'])[0]
# single_year_df['RECI_Kalman'] = kf.em(single_year_df['RECI'], n_iter=5).smooth(single_year_df['RECI'])[0]
# single_year_df['LSWI_Kalman'] = kf.em(single_year_df['LSWI'], n_iter=5).smooth(single_year_df['LSWI'])[0]

single_year_farm2_df['NDVI_Kalman_f2'] = kf.em(single_year_farm2_df['NDVI'], n_iter=5).smooth(single_year_farm2_df['NDVI'])[0]

Enter the start year for analysis (2019-2023): 2019
Enter the end year for analysis (2019-2023): 2023


In [ ]:
# Spline Smoothing (Univariate Spline)
spline_ndvi = UnivariateSpline(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['NDVI'], s=0.5)
spline_ndvi_f2 = UnivariateSpline(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), single_year_farm2_df['NDVI'], s=0.5)

single_year_df['NDVI_Spline'] = spline_ndvi(single_year_df['Date'].map(pd.Timestamp.toordinal))
single_year_farm2_df['NDVI_Spline_f2'] = spline_ndvi_f2(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal))

# BSpline Smoothing
tck = splrep(single_year_df['Date'].map(pd.Timestamp.toordinal), single_year_df['NDVI'], s=0.65)
bspline_ndvi = splev(single_year_df['Date'].map(pd.Timestamp.toordinal), tck)
single_year_df['NDVI_BSpline'] = bspline_ndvi

tck_f2 = splrep(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), single_year_farm2_df['NDVI'], s=0.65)
bspline_ndvi_f2 = splev(single_year_farm2_df['Date'].map(pd.Timestamp.toordinal), tck_f2)
single_year_farm2_df['NDVI_BSpline_f2'] = bspline_ndvi_f2

# Wavelet Smoothing
def wavelet_smoothing(signal):
    coeffs = pywt.wavedec(signal, 'db1', level=6)
    coeffs[1:] = [pywt.threshold(c, value=0.5, mode='soft') for c in coeffs[1:]]
    smoothed_signal = pywt.waverec(coeffs, 'db1')
    return smoothed_signal[:len(signal)]  # Trim to match original length

single_year_df['NDVI_Wavelet'] = wavelet_smoothing(single_year_df['NDVI'])
single_year_farm2_df['NDVI_Wavelet_f2'] = wavelet_smoothing(single_year_farm2_df['NDVI'])

# Fourier Transform Smoothing (Optional, commented out)
# def fourier_smoothing(signal, threshold=0.1):
#     fft_signal = fft(signal)
#     fft_signal[np.abs(fft_signal) < threshold * np.max(np.abs(fft_signal))] = 0
#     return np.real(ifft(fft_signal))

# single_year_df['NDVI_Fourier'] = fourier_smoothing(single_year_df['NDVI'])

# Visualization
fig = go.Figure()

# Add original NDVI values
fig.add_trace(go.Scatter(x=ndvi_df_farm1['Date'], y=ndvi_df_farm1['NDVI'], name = 'NDVI F1 (>0.15)'))
fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df['NDVI'], mode='markers+lines', name='NDVI F1 (Filtered)'))
fig.add_trace(go.Scatter(x=ndvi_df_farm2['Date'], y=ndvi_df_farm2['NDVI'], name = 'NDVI F2 (>0.15)'))
fig.add_trace(go.Scatter(x=single_year_farm2_df['Date'], y=single_year_farm2_df['NDVI'], mode='markers+lines', name='NDVI F2 (Filtered)'))

# Add smoothed NDVI values for each smoothing technique
smoothing_methods = ['SG', 'MA', 'EMA', 'Gaussian', 'LOESS', 'Kalman', 'Spline', 'Wavelet', 'BSpline']  # 'Fourier' if using Fourier
colors = {'SG': 'orange', 'MA': 'purple', 'EMA': 'cyan', 'Gaussian': 'pink', 'LOESS': 'brown', 'Kalman': 'red', 'Spline': 'magenta', 'Wavelet': 'teal', 'BSpline': 'blue'}  # Add color for Fourier if used

for method in smoothing_methods:
    fig.add_trace(go.Scatter(x=single_year_df['Date'], y=single_year_df[f'NDVI_{method}'], mode='lines', name=f'NDVI F1 ({method})', line=dict(color=colors[method])))
    fig.add_trace(go.Scatter(x=single_year_farm2_df['Date'], y=single_year_farm2_df[f'NDVI_{method}_f2'], mode='lines', name=f'NDVI F2 ({method})', line=dict(color=colors[method])))

# Define the year dynamically from the data or user input
year = single_year_df['Date'].dt.year.iloc[0]  # Assuming all dates are in the same year, or use a specific year

# Define the specific periods for shading
wet_season_start = f'{year}-04-01'
wet_season_end = f'{year}-10-31'

rainfall_peak_start = f'{year}-08-01'
rainfall_peak_end = f'{year}-09-30'

dry_season_start = f'{year}-11-01'
dry_season_end = f'{year+1}-03-31'  # Extends into the next year

# Add shaded region for wet season (April - October)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=wet_season_start,
    y0=0,
    x1=wet_season_end,
    y1=1,
    fillcolor="LightGreen",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for rainfall peak (August - September)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=rainfall_peak_start,
    y0=0,
    x1=rainfall_peak_end,
    y1=1,
    fillcolor="LightCoral",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# Add shaded region for dry season (November - March)
fig.add_shape(
    type="rect",
    xref="x",
    yref="paper",
    x0=dry_season_start,
    y0=0,
    x1=dry_season_end,
    y1=1,
    fillcolor="Yellow",
    opacity=0.2,
    layer="below",
    line_width=0,
)

# Update layout with title and labels
fig.update_layout(
    title=f'NDVI for {year}-{year+1} with Various Smoothing Techniques which the values <0.15 are dropped and filtered the peaks only',
    xaxis_title='Date',
    yaxis_title='NDVI Value',
    legend_title_text='NDVI Smoothing Methods'
)

# Show the figure
fig.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

